In [1]:
# from comet_ml import Experiment
# import wandb

# Modeling Cross Section Data with KNN, DT, and Random Forest

In [1]:
COMMET_PROJECT_NAME = "ml-scikit-exfor"
WANDB_PROJECT_NAME = COMMET_PROJECT_NAME
DATASET = 'B0'
VERSION = '_' + "v4" # args.version
DATASET_DICT = {"B0":0, "B1":1, "B2":2}
NORMALIZER_TYPE = "maxabs"
MT_STRATEGY = "one_hot"
TRAIN_FRACTION = 0.9
K_LIST = [1, 2, 3] # args.k_list
DISTANCE_METRIC = ["euclidean", "manhattan"]

In [25]:
import pandas as pd
import itertools
import numpy as np
import os
from joblib import dump
import time
from sklearn.neighbors import KNeighborsRegressor
import sys
from sklearn.model_selection import train_test_split

# This allows us to import the nucml utilities
sys.path.append("../../..")

import nucml.exfor.data_utilities as exfor_utils 
import nucml.datasets as nuc_data 
import nucml.general_utilities as gen_utils

In [26]:
import importlib
importlib.reload(exfor_utils)
importlib.reload(nuc_data)
importlib.reload(gen_utils)
print("Finish re-loading scripts.")

Finish re-loading scripts.


# Loading EXFOR Data

In [3]:
df, x_train, x_test, y_train, y_test, to_scale, scaler = nuc_data.load_exfor(
    pedro=True, basic=DATASET_DICT[DATASET], frac=1-TRAIN_FRACTION, scaling_type=NORMALIZER_TYPE, mt_coding=MT_STRATEGY)

INFO:root: MODE: neutrons
INFO:root: LOW ENERGY: True
INFO:root: LOG: True
INFO:root: BASIC: 0
INFO:root: SCALER: MAXABS
INFO:root:Reading data from C:/Users/Pedro/Desktop/ML_Nuclear_Data/EXFOR/CSV_Files\EXFOR_neutrons\EXFOR_neutrons_MF3_AME_no_RawNaN.csv
INFO:root:Data read into dataframe with shape: (4184115, 8)
INFO:root:Splitting dataset into training and testing...
INFO:root:Normalizing dataset...
INFO:root:Fitting new scaler.


# Newly Measured Chlorine (n,p) Unseen Data 

These data points are not in the current EXFOR data package and will be used to test the new modeling capabilities.

In [27]:
new_cl_data_kwargs = {"Z":17, "A":35, "MT":103, "log":True, "scale":True, "scaler":scaler, "to_scale":to_scale, "one_hot":True}
new_cl_data = exfor_utils.load_newdata("../../../EXFOR/New_Data/Chlorine_Data/new_cl_np.csv", df, **new_cl_data_kwargs)
new_cl_data.head()

INFO:root:Extracting samples from dataframe.
INFO:root:EXFOR extracted DataFrame has shape: (215, 40)
INFO:root:Scaling dataset...
INFO:root:EXFOR extracted DataFrame has shape: (12, 40)


103


,Energy,Data,Z,N,A,MT_1,MT_101,MT_102,MT_103,MT_104,...,MT_33,MT_4,MT_41,MT_51,MT_9000,MT_9001,Center_of_Mass_Flag_Center_of_Mass,Center_of_Mass_Flag_Lab,Element_Flag_I,Element_Flag_N
0,6.383815,-1.779892,0.171717,0.115385,0.137795,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
1,6.383815,-1.707744,0.171717,0.115385,0.137795,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
2,6.401401,-1.583359,0.171717,0.115385,0.137795,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
3,6.401401,-1.590067,0.171717,0.115385,0.137795,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
4,6.411620,-1.350665,0.171717,0.115385,0.137795,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0


In [28]:
a = [list(K_LIST), list(DISTANCE_METRIC)]
total_num_iterations = len(list(itertools.product(*a)))
loop_number = 1 # ORIGINAL INDENT
for k_number in K_LIST:   
    for distance_metric in DISTANCE_METRIC:

        GROUP_NAME = "KNN_{}".format(DATASET) # CHANGED
        RUN_NAME = 'k{}_{}_{}_{}_{}{}'.format(
            k_number, distance_metric, NORMALIZER_TYPE, MT_STRATEGY, DATASET, VERSION) 

        
        if RUN_NAME in os.listdir("E:ML_Models_EXFOR/KNN_{}/".format(DATASET)):
            print("Duplicate Training. Skipping")
            loop_number = loop_number + 1
            continue

        print("Iteration {}/{}".format(loop_number, total_num_iterations))

Iteration 1/6
Iteration 1/6
Iteration 1/6
Iteration 1/6
Iteration 1/6
Iteration 1/6
